有時候我們會遇到類別不平衡的情形。為了說明，我們用乳癌數據集生成一個資料集，由90%的良性與10%的惡性所組成。(本篇code有誤!)

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

df = pd.read_csv('wdbc.data', header=None) # 讀取乳癌數據集
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# 以8:2的比例製造測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1) 

pipe_svc = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        SVC(C=100.0, gamma=0.001, kernel='rbf', random_state=1)) # 參數是上一節中用巢狀交叉驗證找的

pipe_svc.fit(X_train, y_train)
y_pred = pipe_svc.predict(X_test)
print('Test Accuracy: %.3f' % pipe_svc.score(X_test, y_test))

X_imb = np.vstack((X[y == 0], X[y == 1][:40]))
y_imb = np.hstack((y[y == 0], y[y == 1][:40]))

y_pred = np.zeros(y_imb.shape[0])
np.mean(y_pred == y_imb) * 100

Test Accuracy: 0.965


D:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

如此一來就算我們瞎猜，都有90%的準確率!為了避免這種情形，我們可以用以下幾種策略:
1. 在fitting時給予夠大的懲罰。
2. 取後放回。對樣本個數少的類別，提高採樣率;樣本個數多的類別減少之(即濫竽充數也)。
3. 移除較多的樣本

In [4]:
from sklearn.utils import resample

print('Number of class 1 samples before:', X_imb[y_imb == 1].shape[0])

X_upsampled, y_upsampled = resample(X_imb[y_imb == 1],
                                    y_imb[y_imb == 1],
                                    replace=True,
                                    n_samples=X_imb[y_imb == 0].shape[0],
                                    random_state=123)

print('Number of class 1 samples after:', X_upsampled.shape[0])

Number of class 1 samples before: 0
Number of class 1 samples after: 0


In [5]:
X_bal = np.vstack((X[y == 0], X_upsampled))
y_bal = np.hstack((y[y == 0], y_upsampled))

In [6]:
y_pred = np.zeros(y_bal.shape[0])
np.mean(y_pred == y_bal) * 100

nan